In [1]:
import mlflow
import datetime as dt
from model1 import *
from preprocess import *
# from train import *

In [3]:
ticker = 'AAPL'
start_date = '2023-01-01'
end_date = '2024-01-01'
download_start = '2022-11-01'

In [ ]:
def save_model(self, file_path):
        with open(file_path, 'wb') as file:
            pickle.dump(self.q_table, file)

In [12]:
train(ticker, start_date, end_date, download_start, './artifacts/model.pkl')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Episode 1/249, Reward: 3679.7418212890625
Episode 2/249, Reward: 4400.117378234863
Episode 3/249, Reward: 2944.8074645996094
Episode 4/249, Reward: 1994.259536743164
Episode 5/249, Reward: 1317.198471069336
Episode 6/249, Reward: 4059.2222442626953
Episode 7/249, Reward: 2969.7888259887695
Episode 8/249, Reward: 3614.0705337524414
Episode 9/249, Reward: 3354.7288818359375
Episode 10/249, Reward: 3049.4403533935547
Episode 11/249, Reward: 1936.5672912597656
Episode 12/249, Reward: 1408.640739440918
Episode 13/249, Reward: 1497.7459869384766
Episode 14/249, Reward: 1832.7748413085938
Episode 15/249, Reward: 4070.078369140625
Episode 16/249, Reward: 2985.564712524414
Episode 17/249, Reward: 2927.3547744750977
Episode 18/249, Reward: 2886.7154998779297
Episode 19/249, Reward: 3276.879280090332
Episode 20/249, Reward: 1301.520736694336
Episode 21/249, Reward: 158.1470947265625
Episode 22/249, Reward: 3845.089340209961
Episode 23/249, Reward: 4665.7855224609375
Episode 24/249, Reward: 251.84

In [2]:
def load_and_get_action(ticker, target_date):
    date = dt.datetime.strptime(target_date, "%Y-%m-%d")
    start_date = (date - dt.timedelta(days=60)).strftime('%Y-%m-%d')
    end_date = (date + dt.timedelta(days=1)).strftime('%Y-%m-%d')
    data = yf.download(ticker, start=start_date, end=end_date).reset_index()
    data['macd'] = MACD(data['Close']).macd()
    data['obv'] = calculate_obv(data)
    target_date_data = data[data['Date'] == target_date]

    if len(target_date_data) == 0:
        msg = "Market is close"
        return msg, msg, msg

    state = np.array([
        target_date_data['Close'],
        target_date_data['Volume'],
        target_date_data['macd'],
        target_date_data['obv']
    ])
    env_start = (date - dt.timedelta(days = 425)).strftime('%Y-%m-%d')
    env_end = date.strftime('%Y-%m-%d')
    env_real_start = (date - dt.timedelta(days = 365)).strftime('%Y-%m-%d')
    env = StockTradingEnv(ticker, env_start, env_end, env_real_start)
    agent = QLearningAgent(env)
    with open('./artifacts/model.pkl', 'rb') as file:
        agent.q_table = pickle.load(file)

    action = agent.choose_action(state)
    action_mapping = {0: 'Sell', 1: 'Hold', 2: 'Buy'}
    
    return action_mapping[action], state, target_date_data

In [35]:
result = load_and_get_action('AAPL', '2024-06-25')
print(f"Today's action: {result[0]}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Today's action: Buy
